# #130 RC Sections
<i>Generate RC Section Definitions for spreadsheet inputs.</i>
***

In [ ]:
# xlsx file containing the definitions
file_name = "Datafiles/RC Section Definitions.xlsx"

#### Connect to LUSAS and get the database

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
import pandas as pd
import numpy as np
import re
from shared.LPI_21_1 import *
lusas = get_lusas_modeller()

if not lusas.existsDatabase():
    raise Exception("A model file must be open in which to create the RC Section Definitions")

db = lusas.getDatabase()
model_units = db.getModelUnits()
model_length_factor = model_units.getLengthFactor()
print(model_length_factor)

Helper functions

In [ ]:
# Add a row to the reinforcement definition representing a layer of reinforcement corresponding to a face of the section
def add_rebar_row(attr:IFReinforcementSection, row:int, faceIndex:int, layerIndex:int, barsCount:int, start:int, end:int, gap:float, dia:float, end_bars:float):
    attr.setReinforcementValue("rebar", row, "faceIndex", faceIndex)
    attr.setReinforcementValue("rebar", row, "layerIndex", layerIndex)
    attr.setReinforcementValue("rebar", row, "barsCount", barsCount)
    attr.setReinforcementValue("rebar", row, "start", start)
    attr.setReinforcementValue("rebar", row, "end", end)
    attr.setReinforcementValue("rebar", row, "gap", gap)
    attr.setReinforcementValue("rebar", row, "barsDiameter", dia)
    attr.setReinforcementValue("rebar", row, "altBarsDiameter", -1.0)
    attr.setReinforcementValue("rebar", row, "endBarsDiameter", end_bars)
    attr.setReinforcementValue("rebar", row, "materialRef", "R1")

In [ ]:
# Add all the reinforcement layers specified for the section
def add_all_rebar_layers(range:iter, row:pd.Series, rc_section_attr:IFReinforcementSection, length_factor:float):
    attr_row_count = 0
    alternating_end_bars = -1
    for i in range:
        bars = row[f"Bars{i}"]
        if not isinstance(bars, str) and np.isnan(bars) : 
            alternating_end_bars = -1
            continue

        bar_layers = bars.split("|")
        for j, layer in enumerate(bar_layers):

            if not re.match("\\d+x\\d+", layer):
                print(f"Bars {bar_layers[j]} specified in wrong format, must be of form 5x25")
                continue
            
            bar_count = int(bar_layers[j].split("x")[0])
            bar_diameter = float(bar_layers[j].split("x")[1]) * length_factor

            add_rebar_row(rc_section_attr, row=attr_row_count, faceIndex=i, layerIndex=j+1, barsCount=bar_count, start=-1, end=-1, gap=0.0, dia=bar_diameter, end_bars=alternating_end_bars)
            attr_row_count+=1
        alternating_end_bars = -2 if alternating_end_bars == -1 else -1
   

In [ ]:
def create_reinforcement_section_attributes(para_section_utility: IFParametricSection, range:iter, row:pd.Series, length_factor:float):
    # Create the geometric attribute which takes its shape from the utility but provides additional transformations such as eccentricity
    geom_section_attr = db.createGeometricLine(row.Name)
    geom_section_attr.setFromLibrary("Utilities", "", para_section_utility.getName(), 0, 0, 0)
    # Create a Reinforcement utility based on the shape of the geometric section attribute
    rc_section_attr = db.createReinforcementSection(f"{row.Name} - Section Reinforcement").setGeometricAttribute(geom_section_attr)
    rc_section_attr.setReinforcementValue("face", 0, "faceIndex", "All")
    rc_section_attr.setReinforcementValue("face", 0, "actualCover", row.Cover*length_factor)
    rc_section_attr.setReinforcementValue("face", 0, "linkAllowance", 0.0)
    rc_section_attr.setReinforcementValue("face", 0, "allowableCrackWidth", 0.0)
    rc_section_attr.setReinforcementValue("face", 0, "nominalCover", -1.0)
    # Add all the reinforcement layers specified for the section
    add_all_rebar_layers(range, row, rc_section_attr, length_factor)
    # Link the geometric attribute with the associated reinforcement
    rebar_line_attr = db.createReinforcementLine(f"{row.Name} - Member Reinforcement")
    rebar_line_attr.setSegmentValue("Longitudinal", 0, "reinforcementSection", rc_section_attr)
    rebar_line_attr.setSegmentValue("Longitudinal", 0, "stretch", True)
    geom_section_attr.setReinforcement(rebar_line_attr)

In [ ]:
def get_unitset_length_factor(name:str)->IFUnitSet:
    '''Helper function to calculate a length factor to convert the given unit set to the database units'''
    try:
        return model_length_factor / lusas.getUnitSet(name).getLengthFactor()
    except:
        print(f"Warning unrecognised units '{name}'")
        return 1.0

#### Create model attributes as defined in the spereadsheet

In [ ]:
# Create the reinforced section definition specified in the Rectangular definitions sheet
df_rectangles = pd.read_excel(file_name, sheet_name="Rectangular")

for i, row in df_rectangles.iterrows():
    # Length factor from definition units to database units
    l = get_unitset_length_factor(row.Units)
    # Create a parametric section utility
    para_section_utility = db.createParametricSection(f'{row.Name} Utility')
    para_section_utility.setType("Rectangular Solid")
    para_section_utility.setDimensions(['B', 'D'], [row.B*l, row.D*l])
    # Rectangular Sections have 4 faces, starting at the bottom going anticlockwise around the section.
    create_reinforcement_section_attributes(para_section_utility, range(1,5), row, l)

In [ ]:
# Create the reinforced section definition specified in the Tee definitions sheet
df_Tees = pd.read_excel(file_name, sheet_name="Tee")

for i, row in df_Tees.iterrows():
    # Length factor from definition units to database units
    l = get_unitset_length_factor(row.Units)
    # Create a parametric section utility
    para_section_utility = db.createParametricSection(f'{row.Name} Utility')
    para_section_utility.setType("T")
    para_section_utility.setDimensions(['B', 'D', 'tf', 'tw', 'r'], [row.B*l, row.D*l, row.tf*l, row.tw*l, row.r*l])
    # Tee Sections have 8 faces, starting at the bottom of the web going anticlockwise around the section.
    create_reinforcement_section_attributes(para_section_utility, range(1,9), row, l)

In [ ]:
# Create the reinforced section definition specified in the circular definitions sheet
df_circular = pd.read_excel(file_name, sheet_name="Circular")

for i, row in df_circular.iterrows():
    # Length factor from definition units to database units
    l = get_unitset_length_factor(row.Units)
    # Create a parametric section utility
    para_section_utility = db.createParametricSection(f'{row.Name} Utility')
    para_section_utility.setType("Circular Solid")
    para_section_utility.setDimensions(['D'], [row.D*l])
    # Circular Sections have 1 face
    create_reinforcement_section_attributes(para_section_utility, [1], row, l)